# StableTuner v0.1.0 for Linux

Original version https://github.com/devilismyfriend/StableTuner
<br><br>
Testing on docker image dnwalkup/cuda:116-cudnn8-devel-u2004:
<br>
* Ubuntu 20.04, Nvidia CUDNN 8, CUDA 1.16 <br>
* PIP'd - gDown, wget, ftfy, OmegaConf, tqdm, tensorboard, transfomers, triton, pillow, iPython, pycuda, ipywidgets, jupyterlab <br>
* APT'd - zip, unzip, rename, python3, python3-apt, python3-diskutils, python3-pip <br>

### Set Global Variables and Imports

In [ ]:
google_data_cap = False
#Is google capping your bandwidth and not letting you download your files? If so, this needs to be true.

goog_dataset_url = ""
#Google url for your training dataset (e.g. instance images or user images). Use zip file.

other_dataset_url = ""
#Hugging face, dropbox, etc. if google is capping your bandwidth for your training dataset (e.g. instance images or user images). Use zip file.

# Add class images feature
#goog_regimg_url = ""
#Google url for your regularization images (e.g. class images). Use zip file.

#other_regimg_url = ""
#Hugging face, dropbox, etc. if google is capping your bandwith for your regularization images (e.g. class images). Use zip file.


from subprocess import getoutput
from IPython.display import clear_output

clear_output()
print('Imports complete')

### Install Apt Dependencies for Linux

In [ ]:
!apt update
!apt install -y git
!apt install -y nvidia-cuda-toolkit
!apt install -y zip
!apt install -y unzip

clear_output()
print('Apt installs complete')

### Install Pip Dependencies

In [ ]:
!python3 -m pip install scikit-image
!python3 -m pip install scipy
!python3 -m pip install numpy
!python3 -m pip install requests
!python3 -m pip install bitsandbytes
!python3 -m pip install accelerate==0.15.0
!python3 -m pip install huggingface
!python3 -m pip install albumentations
!python3 -m pip install opencv-python
!python3 -m pip install einops
!python3 -m pip install pytorch_lightning
!python3 -m pip install bitsandbytes==0.35.0
!python3 -m pip install safetensors
!python3 -m pip install diffusers

clear_output()
print('Pip installs complete')

### Install xFormers and Final Dependencies

In [ ]:
GPU_CardName = getoutput('nvidia-smi --query-gpu=name --format=csv,noheader')

if '3090' in GPU_CardName:
    !python3 -m pip install https://huggingface.co/dnwalkup/xformers-precompiles/resolve/main/RTX3090-xf14-cu116-py38/xformers-0.0.14.dev0-cp38-cp38-linux_x86_64.whl
    !python3 -m pip uninstall -y torch torchvision
    !python3 -m pip install https://download.pytorch.org/whl/cu116/torch-1.12.1%2Bcu116-cp38-cp38-linux_x86_64.whl
    !python3 -m pip install https://download.pytorch.org/whl/cu116/torchvision-0.13.1%2Bcu116-cp38-cp38-linux_x86_64.whl
    !python3 -m pip install timm
    clear_output()
    print('xFormers and pyTorch installs complete')
elif 'A5000' in GPU_CardName:
    !python3 -m pip install https://huggingface.co/dnwalkup/xformers-precompiles/resolve/main/A5000-xf14-cu116-py38/xformers-0.0.14.dev0-cp38-cp38-linux_x86_64.whl
    !python3 -m pip uninstall -y torch torchvision
    !python3 -m pip install https://download.pytorch.org/whl/cu116/torch-1.12.1%2Bcu116-cp38-cp38-linux_x86_64.whl
    !python3 -m pip install https://download.pytorch.org/whl/cu116/torchvision-0.13.1%2Bcu116-cp38-cp38-linux_x86_64.whl
    !python3 -m pip install timm
    clear_output()
    print('xFormers and pyTorch installs complete')
else:
    print('No prebuilt xformers available for your card')

### Download Necessary Files

In [ ]:
!mkdir /workspace/scripts
%cd /workspace/scripts

!wget -O trainer.py https://github.com/dnwalkup/StableTuner/raw/main/linux/scripts/trainer.py
!wget -O converters.py https://github.com/dnwalkup/StableTuner/raw/main/linux/scripts/converters.py

%cd /workspace

clear_output()
print('Training files downloaded')

### Download Training Dataset

In [ ]:
if google_data_cap == False:
    !gdown --fuzzy $goog_dataset_url -O dataset.zip
else:
    !wget -O dataset.zip $other_dataset_url

!unzip dataset.zip
!rm dataset.zip         #Delete downloaded zip to save space, comment out with "#" if you want to keep.

clear_output()
print('Instance images downloaded and unzipped')

### Download Regularization Images (SKIP FOR NOW)

In [ ]:
if google_data_cap == False:
    !gdown --fuzzy $goog_regimg_url -O reg_images.zip
else:
    !wget -O reg_images.zip $other_regimg_url

!unzip reg_images.zip
!rm reg_images.zip         #Delete downloaded zip to save space, comment out with "#" if you want to keep.

clear_output()
print('Regularization images downloaded and unzipped')

### Launch Training

In [ ]:
!accelerate launch "--mixed_precision=fp16" "/workspace/scripts/trainer.py" \
    "--disable_cudnn_benchmark" \
    "--use_text_files_as_captions" \
    "--pretrained_model_name_or_path=stabilityai/stable-diffusion-2-1" \
    "--pretrained_vae_name_or_path=" \
    "--output_dir=/workspace/output_model" \
    "--seed=3434554" \
    "--resolution=768" \
    "--train_batch_size=1" \
    "--num_train_epochs=100" \
    "--mixed_precision=fp16" \
    "--use_bucketing" \
    "--use_8bit_adam" \
    "--gradient_checkpointing" \
    "--gradient_accumulation_steps=1" \
    "--learning_rate=3e-6" \
    "--lr_warmup_steps=0" \
    "--lr_scheduler=constant" \
    "--save_latents_cache" \
    "--concepts_list=/workspace/stabletune_concept_list.json" \
    "--num_class_images=0" \
    "--save_every_n_epoch=10" \
    "--train_text_encoder" \
    "--n_save_sample=0" \
    "--sample_height=512" \
    "--sample_width=512" \
    "--dataset_repeats=1"